# TP4

In [ ]:
%matplotlib inline

# Entraîner un classificateur

Ça y est. Vous avez vu comment définir des réseaux de neurones, calculer les pertes et faire
mises à jour des poids du réseau.


## Et les données ?

Généralement, lorsque vous devez traiter des données d'image, de texte, audio ou vidéo,
vous pouvez utiliser des packages Python standard qui chargent des données dans un tableau numpy.
Ensuite, vous pouvez convertir ce tableau en ``torch.*Tensor``.

- Pour les images, des packages tels que Pillow, OpenCV sont utiles
- Pour l'audio, des packages tels que scipy et librosa


Spécifiquement pour la vision, un package appelé
``torchvision``, possède des chargeurs de données pour les ensembles de données courants tels que
ImageNet, CIFAR10, MNIST, etc. et transformateurs de données pour les images, à savoir,
``torchvision.datasets`` et ``torch.utils.data.DataLoader``. C'est très pratique !

Pour ce tutoriel, nous utiliserons l'ensemble de données CIFAR10.
Voir des exemples sur le [lien](https://www.cs.toronto.edu/~kriz/cifar.html)


## Entraîner un classificateur d'images

Nous allons suivre les étapes suivantes dans l'ordre :

1. Charger et normaliser les ensembles de données de formation et de test CIFAR10 à l'aide
   ``torchvision``
2. Définir un réseau neuronal convolutif
3. Définir une fonction de perte
4. Entraîner le réseau sur le training set
5. Testez le réseau sur les données de test


### 1. Load and normalize CIFAR10

En utilisant ``torchvision``, il est très facile de travailler avec Cifar10.


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

La sortie des ensembles de données torchvision sont des images PIL Image à valeurs dans [0, 1].
Nous les transformons en tensors normalisés à valeurs dans [-1, 1].



In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

trainloader et testloader nous permettront d’échantillonner des lots de données.

 <font color='blue'> **Question** : Quelles sont les classes du dataset ? ([link](https://www.cs.toronto.edu/~kriz/cifar.html)) </font>

  <font color='blue'> **Question** : Combien d'images sont dans le training set ? </font>
  
  <font color='blue'> **Question** : Combien d'images sont dans le test set  ? </font>

  <font color='blue'> **Question** : Que fait ```transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))``` ? Comment cela serait-il codé pour des images en noir et blanc ? </font>

Regardons quelques images du training set



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
images.shape

### 2. Define a Convolutional Neural Network



 <font color='blue'> **Question** : Compléter la cellule suivante </font>



In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(__, 120) #TODO
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, __) #TODO

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
net(images)

### 3. Define a Loss function and optimizer




In [ ]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
criterion = nn.CrossEntropyLoss()

### 4. Entraînement du réseau




In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

 <font color='blue'> **Question** : Que fait la cellule précedente ?

See [here](https://pytorch.org/docs/stable/notes/serialization.html)
for more details on saving PyTorch models.

### 5. Test du réseau

Nous avons entraîné le réseau avec 2 passages sur l'ensemble de données de formation.
Vérifions si le réseau a appris quelque chose.



In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

Ensuite, chargeons à nouveau notre modèle enregistré (remarque : sauvegarder et recharger le modèle
n'était pas nécessaire ici, nous l'avons fait uniquement pour illustrer comment procéder) :



In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH))

D'accord, voyons maintenant ce que le réseau neuronal pense de ces exemples ci-dessus :



In [ ]:
outputs = net(images)
print(outputs)

 <font color='blue'> **Question** : Quelles sont les probabilités que la première image du batch se trouve dans chaque classe ? </font>

In [ ]:
_, predicted = torch.max(outputs, 1)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))
print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

Observons la matrice de confusion.

 <font color='blue'> **Question** : Qu'est-ce qui est fait dans la cellule suivante ? </font>

In [ ]:
all_labels = torch.tensor([])
all_predicted = torch.tensor([])
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)

        all_labels = torch.cat((all_labels, labels))
        all_predicted = torch.cat((all_predicted, predicted))


 <font color='blue'> **Question** : Compléter la cellule ci-dessous. </font>

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

t_test = #TODO
t_pred = #TODO


#Classification report
print(classification_report(t_test,t_pred,target_names = classes))

#Confusion matrix
cm = confusion_matrix(t_test,t_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels = classes)
disp.plot()